# Problem Statement 1: League Winner Prediction
## Predict the League Champion/Winner

**Author:** ScoreSight ML Team  
**Date:** 2025-11-12  
**Problem Type:** Classification (League Champion/Position Prediction)

### Dataset
- **File:** `data/data_engineered_league_points.csv`
- **Task:** Predict league winner and final positions
- **Features:** Team statistics, performance metrics (22+ engineered features)
- **Target:** League champion indicator or final position

### Pipeline
1. Load and explore dataset
2. Preprocess features and target
3. Train classification models with hyperparameter tuning
4. Evaluate with cross-validation
5. Save best models to `models/` folder

## 1. Import Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False

try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    LGB_AVAILABLE = False

# For Colab: Base directory is /content/drive/MyDrive/ScoreSight
SCORESIGHT_DIR = '/content/drive/MyDrive/ScoreSight'
DATA_DIR = os.path.join(SCORESIGHT_DIR, 'data')
MODELS_DIR = os.path.join(SCORESIGHT_DIR, 'models')

# Create models directory
Path(MODELS_DIR).mkdir(exist_ok=True, parents=True)

print("[OK] All libraries imported")
print(f"XGBoost: {XGB_AVAILABLE} | LightGBM: {LGB_AVAILABLE}")
print(f"ScoreSight directory: {SCORESIGHT_DIR}")
print(f"Data directory: {DATA_DIR}")

## 2. Load and Explore Data

In [ ]:
# First, let's see what's actually in your Google Drive
print("Checking Google Drive structure...")
print("\n1. Contents of /content/drive/MyDrive/:")
try:
    mydrive_contents = os.listdir('/content/drive/MyDrive/')
    for item in sorted(mydrive_contents):
        item_path = os.path.join('/content/drive/MyDrive/', item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")
except Exception as e:
    print(f"  Error: {e}")

print("\n2. Looking for ScoreSight folder...")
scoresight_path = '/content/drive/MyDrive/ScoreSight'
if os.path.exists(scoresight_path):
    print(f"✅ Found: {scoresight_path}")
    print("\n3. Contents of ScoreSight/:")
    for item in sorted(os.listdir(scoresight_path)):
        item_path = os.path.join(scoresight_path, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")
else:
    print(f"❌ NOT FOUND: {scoresight_path}")
    print("\nPlease check:")
    print("1. Is your Google Drive mounted? (You should see the drive icon)")
    print("2. What is the EXACT name of your ScoreSight folder in Google Drive?")
    print("3. Did you upload to 'My Drive' or somewhere else?")
    
print("\n" + "="*60)
print("Once we find the correct path, we'll load the data.")
print("="*60)

## 3. Prepare Features and Target

In [ ]:
# Identify target column (usually champion or winner)
target_col = None
for col in df.columns:
    if any(x in col.lower() for x in ['champion', 'winner', 'position', 'rank']):
        target_col = col
        break

if target_col is None:
    target_col = df.columns[-1]  # Last column as fallback

print(f"[TARGET] Using column: {target_col}")
print(f"Target distribution:")
print(df[target_col].value_counts())

# Prepare features and target
exclude_cols = [target_col, 'unnamed: 0', 'team', 'season', 'team_name', 'league']
feature_cols = [col for col in df.columns 
               if col not in exclude_cols and df[col].dtype in ['float64', 'int64']]

X = df[feature_cols].copy()
y = df[target_col].copy()

# Handle missing values in features
X = X.fillna(X.mean())

# Encode target if necessary
if y.dtype == 'object':
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"Target classes: {list(le.classes_)}")
else:
    y_encoded = y.values
    le = None

print(f"\n[OK] Features: {len(feature_cols)}")
print(f"[OK] Samples: {len(X)}")
print(f"[OK] Feature matrix shape: {X.shape}")

## 4. Define Pipeline and Hyperparameters

In [ ]:
def create_pipeline(model, use_scaling=True):
    steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaling:
        steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    return Pipeline(steps)

# Define models
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=2000),
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)
}

if XGB_AVAILABLE:
    models['XGBoost'] = xgb.XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss')

if LGB_AVAILABLE:
    models['LightGBM'] = lgb.LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1)

print(f"[OK] {len(models)} models defined")

## 5. Train Models

In [ ]:
results = {}
trained_models = {}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n[{name}] Training...")
    
    pipeline = create_pipeline(model, use_scaling=(name == 'LogisticRegression'))
    pipeline.fit(X, y_encoded)
    
    # Evaluate
    scores = []
    for train_idx, test_idx in cv.split(X, y_encoded):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
        
        temp_pipeline = create_pipeline(model.__class__(**model.get_params()), use_scaling=(name == 'LogisticRegression'))
        temp_pipeline.fit(X_train, y_train)
        y_pred = temp_pipeline.predict(X_test)
        scores.append(f1_score(y_test, y_pred, average='weighted'))
    
    results[name] = {'f1': np.mean(scores), 'f1_std': np.std(scores)}
    trained_models[name] = pipeline
    
    print(f"  F1-weighted: {results[name]['f1']:.4f} (+/- {results[name]['f1_std']:.4f})")

print("\n[OK] All models trained")

## 6. Model Comparison

In [ ]:
comparison_df = pd.DataFrame(results).T.sort_values('f1', ascending=False)
comparison_df.columns = ['F1-Score', 'Std Dev']

print("\n" + "="*60)
print("[RESULTS] Model Comparison")
print("="*60)
print(comparison_df)

best_model_name = comparison_df.index[0]
best_f1 = comparison_df.iloc[0]['F1-Score']
print(f"\n[BEST] {best_model_name} (F1: {best_f1:.4f})")

## 7. Save Models

In [ ]:
models_dir = Path(MODELS_DIR)
models_dir.mkdir(exist_ok=True, parents=True)

for name, model in trained_models.items():
    model_path = models_dir / f"league_winner_{name.lower().replace(' ', '_')}_ps1.joblib"
    joblib.dump(model, model_path)
    print(f"[SAVE] {name} -> {model_path}")

if le is not None:
    le_path = models_dir / "league_winner_encoder_ps1.joblib"
    joblib.dump(le, le_path)
    print(f"[SAVE] Label Encoder -> {le_path}")

# Save summary
summary = {
    'problem_statement': 'League Winner Prediction',
    'timestamp': datetime.now().isoformat(),
    'best_model': best_model_name,
    'best_f1': float(best_f1),
    'n_features': len(feature_cols),
    'n_samples': len(X),
    'models': {name: {'f1': float(results[name]['f1']), 'model_path': str(models_dir / f"league_winner_{name.lower().replace(' ', '_')}_ps1.joblib")} for name in results.keys()}
}

summary_path = models_dir / "league_winner_training_summary_ps1.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n[SAVE] Summary -> {summary_path}")
print("\n[COMPLETE] League Winner Prediction Training Finished")